In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sklearn

from sklearn import decomposition
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

import pandas as pd

# Last ned datasett

In [ ]:
np.random.seed(5)

centers = [[1, 1], [-1, -1], [1, -1]]
iris = datasets.load_iris()

X = iris.data
Y = iris.target

Oversikt over attributtene

In [ ]:
list(iris.feature_names)

Oversikt over klassene

In [ ]:
list(iris.target_names)

# Tren en klassifiseringsmodell på datasettet

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
np.random.seed(0)
indices = np.random.permutation(len(X))
number_of_training_examples = int(0.8*len(X))

X_train=X[indices[:number_of_training_examples]]
Y_train=Y[indices[:number_of_training_examples]]
X_test=X[indices[number_of_training_examples:]]
Y_test=Y[indices[number_of_training_examples:]]

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, Y_train) 
knn.score(X_test, Y_test)


# Feature selection

En enkel måte å redusere dimensjonene til et problem er å droppe de minst viktige attributtene, kalt feature selection. Det er mange måter å gjøre det på. Èn måte er å fjerne alle attributter som har en varians lavere enn et gitt nivå, som i eksempelet under. 

In [ ]:
sel = VarianceThreshold(threshold=0.2)
sel.fit(X)
X_feature_selection = sel.transform(X)
print("Hvilke attributter som var viktige nok til å få være med " + str(sel.get_support()))

# PCA

## Oppgave 1 

Gjør en PCA på datasettet ved hjelp av scikit (http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). 

Reduserer dimensjonen til X slik at den kun har 3 dimensjoner og lagre resultatet i X_pca. 

In [ ]:
pca = decomposition.PCA(n_components=3)
pca.fit(X)
X_pca = pca.transform(X)

# Oppgave 2

PCA-analysen i forrige oppgave skal ha gitt deg en X-matrise som nå kun består av tre features/attributter. Disse tre attributtene er ikke lik noen av de opprinnelige attributtene, men hver nye attributt er en unik kombinasjon av alle de opprinnelige attributtene. I denne oppgaven skal vi se på hvor viktig hver av egenvektorene er for å beskrive datasettet.  

Hent ut en oversikt over hvor stor andel hver egenvektor har for å forklare variansen i datasettet. 

In [ ]:
pca.explained_variance_ratio_

## Oppgave 3

PCA-analysen i forrige oppgave skal ha gitt deg en X-matrise som nå kun består av tre features/attributter. Disse tre attributtene er ikke lik noen av de opprinnelige attributtene, men hver nye attributt er en unik kombinasjon av alle de opprinnelige attributtene. I denne oppgaven skal vi se hvordan vi kan beskrive de nye attributenne ved hjelp av de opprinnelige. 

Hent ut komponentene som PCA-analysen består av.

In [ ]:
pca.components_

Du ser nå (forhåpentligvis) på en 3x4-matrise. Hver rad beskriver en egenvektor (en ny attributt) og innholder fire elementer som representerer de opprinnelige attributene. Hvert element sier deg i hvor stor grad hver egenvektoren beskrives av de opprinnelige attributten. 

# Plot resultatet

In [ ]:
fig = plt.figure(1, figsize=(4, 3))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
plt.cla()

for name, label in [('Setosa', 0), ('Versicolour', 1), ('Virginica', 2)]:
    ax.text3D(X_pca[y == label, 0].mean(),
              X_pca[y == label, 1].mean() + 1.5,
              X_pca[y == label, 2].mean(), name,
              horizontalalignment='center',
              bbox=dict(alpha=.5, edgecolor='w', facecolor='w'))

y = np.choose(Y, [1, 2, 0]).astype(np.float)

ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c=y, cmap=plt.cm.spectral,
           edgecolor='k')

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])

In [ ]:
plt.show()

# Tren en klassifiseringsmodell på det dimensjonsreduserte datasettet

## Oppgave 4

Forventer du et bedre eller dårligere resultat enn modellen du bygget tidligere på hele datasettet? Når ønsker du å bruke PCA?

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
np.random.seed(0)
indices = np.random.permutation(len(X))
number_of_training_examples = int(0.8*len(X))

X_PCA_train=X_pca[indices[:number_of_training_examples]]
Y_train=Y[indices[:number_of_training_examples]]
X_PCA_test=X_pca[indices[number_of_training_examples:]]
Y_test=Y[indices[number_of_training_examples:]]

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_PCA_train, Y_train) 
knn.score(X_PCA_test, Y_test)

# BONUS: Erstatt PCA med Cross Decomposition

Når vi reduserer dimensjonene med PCA ser vi kun på input-dataen (X), og finner de dimensjonene som beskriver datasettet best mulig slik at vi taper minst mulig informasjon når vi reduserer dimensjonene. 

En annen måte å redusere dimensjonene på er å ta i bruk det vi ønsker å predikere (Y) og velge de dimensjonene som beskriver Y best. 

In [ ]:
from sklearn.cross_decomposition import CCA
cca = CCA(n_components=4)
cca.fit(X_train, Y_train)
#X_train_r, Y_train_r = cca.transform(X_train, Y_train)
#X_test_r, Y_test_r = cca.transform(X_test, Y_test)
cca.score(X_test,Y_test)